In [2]:
import mlflow 
from uuid import uuid4

from langchain_core.messages import HumanMessage
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from dotenv import load_dotenv

from mlflow.models.signature import infer_signature
from mlflow_training.agent import create_graph, LangGraphWrapper

load_dotenv()

c:\Users\florian.krempl.AT\Documents\projects\trainings\mlflow-training\.venv\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


True

In [3]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("LLM-Tracing")
mlflow.langchain.autolog()

In [4]:
SYSTEM_MESSAGE = "You are a helpfull Assistant. Use Wikipedia to answer questions about facts and general knowledge."

wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
agent = create_graph(system_message=SYSTEM_MESSAGE, tools=[wikipedia_tool])

In [7]:
session_id = str(uuid4())
config = {
    "configurable": {"thread_id": session_id},
}

response = agent.invoke(input = {"messages": HumanMessage("Who is the current chancellor of Germany? April 2025")}, config=config)
print(response)

{'messages': [HumanMessage(content='Who is the current chancellor of Germany? April 2025', additional_kwargs={}, response_metadata={}, id='88e7af8e-ccd0-4e70-bbcb-c7f5d9d3a143'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_urUqfIdRUqr38sp8XrL3w86K', 'function': {'arguments': '{"query":"Chancellor of Germany"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef'}, id='run-8ddb36a8-9acd-4a5d-aa3e-6b4d8a3843b7-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Chancellor of Germany'}, 'id': 'call_urUqfIdRUqr38sp8XrL3w86K', 'type': 'tool_call'}]), ToolMessage(content='Page: Chancellor of Germany\nSummary: The chancellor of Germany, officially the federal chancellor of the Federal Republic of Germany, is the head of the federal government of Germany. The chancellor is the chief executive of the Federal Cabinet and heads the executive

Trace(request_id=a7ed1b334907436aa91a3b2cd93fbc4c)

In [8]:

model_input = {
    "query": "Who is the current chancellor of Germany? April 2025",
    "session_id": str(uuid4())
}

signature = infer_signature(model_input, response["messages"][-1].content)

with mlflow.start_run() as run:
    # Log the custom LangGraph model.
    mlflow.pyfunc.log_model(
        artifact_path="langgraph_model",
        python_model=LangGraphWrapper(system_message=SYSTEM_MESSAGE, tools=[wikipedia_tool]),
        signature=signature,  # Attach the inferred signature here.
        # Optionally, add a registered model name if you are using the model registry.
        registered_model_name="MyLanggraphAgent"
    )
    mlflow.log_param("system_message", SYSTEM_MESSAGE)
    mlflow.log_param("tools", [wikipedia_tool.name])
    
run_id = run.info.run_id

c:\Users\florian.krempl.AT\Documents\projects\trainings\mlflow-training\.venv\Lib\site-packages\mlflow\pyfunc\__init__.py:3212: UserWarning: An input example was not provided when logging the model. To ensure the model signature functions correctly, specify the `input_example` parameter. See https://mlflow.org/docs/latest/model/signatures.html#model-input-example for more details about the benefits of using input_example.
  color_warning(
Registered model 'MyLanggraphAgent' already exists. Creating a new version of this model...
2025/03/27 16:03:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MyLanggraphAgent, version 6


🏃 View run victorious-fowl-371 at: http://localhost:5000/#/experiments/477751900475547963/runs/5ab80ebbd2e742aea92ea62d6942d69c
🧪 View experiment at: http://localhost:5000/#/experiments/477751900475547963


Created version '6' of model 'MyLanggraphAgent'.


In [6]:
# Replace with your actual run ID or model registry URI.
model_uri = f"runs:/{run_id}/langgraph_model"
loaded_model = mlflow.pyfunc.load_model(model_uri)
loaded_model 

c:\Users\florian.krempl.AT\Documents\projects\trainings\mlflow-training\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


mlflow.pyfunc.loaded_model:
  artifact_path: langgraph_model
  flavor: mlflow.pyfunc.model
  run_id: bf81fb72e0c14ec0827ef8bd382e7618

In [7]:
# Make predictions using the loaded model.

model_input = {
    "query": "Who is the current chancellor of Austria? April 2025",
    "session_id": session_id
}
predictions = loaded_model.predict(model_input)
predictions


'As of April 2025, the current Chancellor of Austria is Christian Stocker, who was sworn in on March 3, 2025.'

Trace(request_id=f1d10205d5b3491ca87b159f65489cf1)